# <center>Data Mining Project 2 Spring semester 2018-2019</center>
## <center>Παναγιώτης Ευαγγελίου &emsp; 1115201500039</center>
## <center>Ευάγγελος Σπίθας &emsp;&emsp;&emsp;&ensp; 1115201500147</center>

___

## Do all the necessary imports for this notebook

In [1]:
import numpy as np
import pandas as pd
import calendar

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## __Read data__

In [2]:
initialDataFrame = pd.read_csv('../data/crime.csv', engine='python')

# initialDataFrame # printToBeRemoved

## __Do some data preparation__

In [3]:
# drop the columns that we don't need
processedDataFrame = initialDataFrame.drop(['Location'], axis=1)

# replace some NaN values
processedDataFrame = processedDataFrame.fillna({'SHOOTING':"Ν"})

processedDataFrame['SHOOTING'].unique()  # printToBeRemoved

array(['Ν', 'Y'], dtype=object)

## __Data Research__

1. #### Multitude of crimes by year, by month, by date and by district

In [4]:
# groupBy year
# yearMultitudeDf = processedDataFrame.groupby(['YEAR']).count() # to be removed
yearMultitudeSeries = processedDataFrame.groupby(['YEAR'])['INCIDENT_NUMBER'].count()

# replace month numbers to month names
monthMultitudeDf = processedDataFrame.copy()
monthMultitudeDf['MONTH'] = monthMultitudeDf['MONTH'].apply(lambda x: calendar.month_abbr[x])

# groupBy month
monthMultitudeSeries = monthMultitudeDf.groupby(['MONTH'])['INCIDENT_NUMBER'].count()
# monthMultitudeDf = monthMultitudeDf.groupby(['MONTH']).count() # to be removed

# groupBy day
# dayMultitudeDf = processedDataFrame.groupby(['DAY_OF_WEEK']).count() # to be removed
dayMultitudeSeries = processedDataFrame.groupby(['DAY_OF_WEEK'])['INCIDENT_NUMBER'].count()

# groupBy district
# districtMultitudeDf = processedDataFrame.groupby(['DISTRICT']).count() # to be removed
districtMultitudeSeries = processedDataFrame.groupby(['DISTRICT'])['INCIDENT_NUMBER'].count()

print("Count by year:")  # printToBeRemoved
print(yearMultitudeSeries)  # printToBeRemoved
print("--------------")  # printToBeRemoved

print("Count by month:")  # printToBeRemoved
print(monthMultitudeSeries)  # printToBeRemoved
print("--------------")  # printToBeRemoved

print("Count by day:")  # printToBeRemoved
print(dayMultitudeSeries)  # printToBeRemoved
print("--------------")  # printToBeRemoved

print("Count by district:")  # printToBeRemoved
print(districtMultitudeSeries)  # printToBeRemoved
print("--------------")  # printToBeRemoved

Count by year:
YEAR
2015     53392
2016     99134
2017    100938
2018     74356
Name: INCIDENT_NUMBER, dtype: int64
--------------
Count by month:
MONTH
Apr    24108
Aug    35137
Dec    23484
Feb    21661
Jan    23625
Jul    34640
Jun    30622
Mar    24156
May    26242
Nov    23685
Oct    26437
Sep    34023
Name: INCIDENT_NUMBER, dtype: int64
--------------
Count by day:
DAY_OF_WEEK
Friday       49758
Monday       46970
Saturday     45969
Sunday       41374
Thursday     47872
Tuesday      47726
Wednesday    48151
Name: INCIDENT_NUMBER, dtype: int64
--------------
Count by district:
DISTRICT
A1     36735
A15     6663
A7     13634
B2     51288
B3     36400
C11    43817
C6     24190
D14    20632
D4     43338
E13    17981
E18    17825
E5     13543
Name: INCIDENT_NUMBER, dtype: int64
--------------
